# Dissecting Spotify Valence
In this assignment i will try to dissect Spotify's Valence metric.

---

> Iliadis Viktoras, Undergraduate Student <br />
> Department of Management Science and Technology <br />
> Athens University of Economics and Business <br />
> iliadisviktoras@gmail.com

In this assignment we will try to untangle the mystery behind spotify's ( originaly Echo Nest's ) valence metric that indicates the emotional impact of a song , particulary it's ability to make you happy ( positive valence ) or sad ( negative valence) by listening to it . 

https://machinelearningmastery.com/calculate-feature-importance-with-python/ εδω εξηγεί για το μοντέλο πως προβλεπω ποια feature επαιξαν ρολο , οπως ζηταει στο προτελευταίο 

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

import glob
import re
from datetime import datetime

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

We are going to read the data we extracted from denodo and the Spotify API 


In [9]:
Spotify_Data= "https://github.com/IliadisVictor/Spotify-Valence-Prediction/blob/main/TrackData/TrackFeatures.csv"
Tracksdf = pd.read_csv(Spotify_Data)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 149, saw 2


In [8]:
Tracksdf

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,Timbre3,Timbre4,Timbre5,Timbre6,Timbre7,Timbre8,Timbre9,Timbre10,Timbre11,Timbre12
0,5aAx2yezTd8zXrkmtKl66Z,0.681,0.594,7,-7.028,1,0.2820,0.16500,0.000003,0.1340,...,-25.474662,-8.497975,11.283421,-9.608518,2.480847,-3.206591,1.009621,6.001180,-18.966575,-1.065181
1,5knuzwU65gJK7IF5yJsuaW,0.720,0.763,9,-4.068,0,0.0523,0.40600,0.000000,0.1800,...,47.688657,13.829239,41.981918,-11.063666,-1.380892,1.131578,-8.356399,4.714077,-12.080945,0.713747
2,7BKLCZ1jbUBVqRi2FVlTVw,0.748,0.524,8,-5.599,1,0.0338,0.41400,0.000000,0.1110,...,-7.485401,-4.112776,48.958689,-12.356689,-12.970229,-4.290602,-11.967572,-1.022499,-15.331625,1.318928
3,3NdDpSvN911VPGivFlV5d0,0.735,0.451,0,-8.374,1,0.0585,0.06310,0.000013,0.3250,...,-8.822800,-14.291958,0.926185,-8.242187,19.188872,8.583830,-1.528432,3.469480,-15.985839,-1.124039
4,78rIJddV4X0HkNAInEcYde,0.670,0.838,0,-4.031,1,0.0362,0.06040,0.000611,0.1590,...,-5.129730,-4.067110,38.045295,-30.086439,-15.993999,2.183294,-6.271883,9.430749,-11.572217,-6.324930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13875,3aQV6E290SJvasp2VOgvBz,0.810,0.690,10,-7.841,0,0.2700,0.03730,0.000000,0.1060,...,-1.869886,12.467732,26.344576,5.268833,-10.892086,3.889907,-14.755216,10.926393,-21.195313,4.200773
13876,43YQDj1KOPMFAUcTEPKy4p,0.942,0.531,1,-7.510,1,0.2150,0.00236,0.000106,0.0812,...,-68.302975,-7.247598,19.064231,-3.791177,8.428280,4.029805,-2.629264,-1.005028,-25.238031,-6.155557
13877,1985nAkGJf1oOrgHCWftoN,0.762,0.679,4,-7.595,0,0.1920,0.04260,0.000000,0.1860,...,-23.941283,-2.921593,34.689293,-8.483353,1.907181,-6.333559,-13.717882,1.212293,-21.931981,-6.245237
13878,7ntUbTIwmwOD6ygCMRZT6e,0.774,0.836,9,-2.323,0,0.1520,0.28800,0.000024,0.0824,...,25.482772,12.983483,34.628555,-14.490607,6.384560,-3.158931,-8.294271,0.330900,-10.555857,3.710331
